In [1]:
import pandas as pd
import folium
import geopandas as gpd

path = "gadm41_VNM_3.shp"
gdf_vn = gpd.read_file(path)
gdf_vn.crs = {'init': 'epsg:4326'}

gdf_city_district = gdf_vn.groupby(['NAME_1', 'NAME_2'])['geometry'].apply(lambda x: x.unary_union).reset_index()
gdf_city_district.columns = ['city', 'district', 'geometry']

/Users/macbookpro/miniconda3/lib/python3.11/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [2]:
gdf_city_district_ward = gdf_vn.groupby(['NAME_1', 'NAME_2', 'NAME_3'])['geometry'].apply(lambda x: x.unary_union).reset_index()
gdf_city_district_ward.columns = ['city', 'district', 'ward', 'geometry']

In [3]:
gdf_city_district

,city,district,geometry
0,An Giang,An Phú,"POLYGON ((105.08266 10.76905, 105.07978 10.772..."
1,An Giang,Châu Phú,"POLYGON ((105.11746 10.43680, 105.11516 10.437..."
2,An Giang,Châu Thành,"POLYGON ((105.28348 10.36473, 105.28017 10.364..."
3,An Giang,Châu Đốc,"POLYGON ((105.10498 10.63827, 105.08154 10.614..."
4,An Giang,Chợ Mới,"POLYGON ((105.42522 10.43894, 105.40791 10.439..."
...,...,...,...
705,Đồng Tháp,Sa Đéc,"POLYGON ((105.77522 10.27055, 105.77191 10.272..."
706,Đồng Tháp,Tam Nông,"POLYGON ((105.50705 10.66007, 105.49598 10.663..."
707,Đồng Tháp,Thanh Bình,"POLYGON ((105.39368 10.56887, 105.38641 10.571..."
708,Đồng Tháp,Tháp Mười,"POLYGON ((105.75810 10.49022, 105.73466 10.495..."


In [4]:
geo_data = pd.read_csv('discrepancy_geo_data.csv')

In [5]:
geo_data =  geo_data.drop(columns=['city', 'district', 'geometry'])

In [6]:
merged_data =  gdf_city_district.merge(geo_data, left_on=['city', 'district'], right_on=['to_city', 'to_district'], how='right')

In [7]:


m = folium.Map(location=[10.7704143,106.6911201], tiles='cartodbpositron', zoom_start=12)
cp = folium.Choropleth(geo_data=merged_data.__geo_interface__, 
           data=merged_data['discrepancy'], 
#            columns=['city', 'cash_ratio'],
           key_on="feature.id", 
           fill_color='YlGnBu', 
           fill_opacity=0.3,
           legend_name='Discrepancy between Crow Flies and Google Map'
          ).add_to(m)

In [8]:
from folium.features import DivIcon

folium.GeoJsonTooltip(['city', 'district', 'discrepancy', 'num_order']).add_to(cp.geojson)
folium.LayerControl().add_to(m)

for i, row in merged_data.iterrows():
    label = f"{row['fromDistrict']}"
    if row['fromDistrict'] in ["Quận 1"]:
        folium.map.Marker(location=[row['geometry'].centroid.y, row['geometry'].centroid.x],
                          icon=DivIcon(
                              icon_size=(100,30),
                              icon_anchor=(10,10),
                              html=f'<div style="font-size: 8pt">{label}</div>',
                          )
                         ).add_to(m)
    elif row['fromDistrict'] in ["Quận 8"]:
        folium.map.Marker(location=[row['geometry'].centroid.y, row['geometry'].centroid.x],
                          icon=DivIcon(
                              icon_size=(100,30),
                              icon_anchor=(20,25),
                              html=f'<div style="font-size: 8pt">{label}</div>',
                          )
                         ).add_to(m)
    elif row['fromDistrict'] in ["Bình Thạnh"]:
        folium.map.Marker(location=[row['geometry'].centroid.y, row['geometry'].centroid.x],
                          icon=DivIcon(
                              icon_size=(100,30),
                              icon_anchor=(60,0),
                              html=f'<div style="font-size: 8pt">{label}</div>',
                          )
                         ).add_to(m)
    else:
        folium.map.Marker(location=[row['geometry'].centroid.y, row['geometry'].centroid.x],
                          icon=DivIcon(
                              icon_size=(100,30),
                              icon_anchor=(20,14),
                              html=f'<div style="font-size: 8pt">{label}</div>',
                          )
                         ).add_to(m)

In [9]:
m

In [10]:
import requests
import json
def get_coordinates_from_way_overpass(way_id):
    overpass_url = "http://overpass-api.de/api/interpreter"
    overpass_query = f"""
    [out:json];
    way({way_id});
    convert item ::=::,::geom=geom(),_osm_type=type();
    out geom;
    """
    response = requests.get(overpass_url, 
                            params={'data': overpass_query})
    data = response.json()
    result = data['elements'][0]['geometry']['coordinates']
    return result

In [11]:
kenh_tau_hu = get_coordinates_from_way_overpass(510783928)
kenh_te = get_coordinates_from_way_overpass(289925740)
kenh_doi = get_coordinates_from_way_overpass(510783929)
kenh_ben_nghe = get_coordinates_from_way_overpass(289925742)
song_sai_gon = get_coordinates_from_way_overpass(708678320)
song_sai_gon_2 = get_coordinates_from_way_overpass(708678316)

In [12]:
from shapely.geometry import LineString

list_rivers = [kenh_tau_hu, kenh_te, kenh_doi, kenh_ben_nghe, song_sai_gon, song_sai_gon_2]
for river in list_rivers:
    new_reversed_list = []
    for i in river:
        new_reversed_list.append((i[-1], i[-2]))
    geom = LineString(new_reversed_list)
    gdf = gpd.GeoDataFrame(geometry=[geom])
    old_tuple = gdf['geometry'].apply(lambda x: list(x.coords))[0]
    folium.PolyLine(locations=old_tuple, color='red', weight=3).add_to(m)

In [13]:
m

In [14]:
top_discrepancy_pair = pd.read_csv('top_discrepancy_pair.csv')
merged_data_to_district =  gdf_city_district[gdf_city_district.city=='Hồ Chí Minh'].merge(top_discrepancy_pair, left_on=['district'], right_on=['to_district'], how='right')
merged_data_to_district =  merged_data_to_district.rename(columns={'geometry':'to_geometry'}).drop(columns=['city', 'district'])

merged_data_from_district =  gdf_city_district[gdf_city_district.city=='Hồ Chí Minh'].merge(merged_data_to_district, left_on=['district'], right_on=['fromDistrict'], how='right')
merged_data_from_district =  merged_data_from_district.rename(columns={'geometry':'from_geometry'}).drop(columns=['city', 'district'])



In [15]:
merged_data_from_district['from_to_district'] = merged_data_from_district.fromDistrict + " - " + merged_data_from_district.to_district

In [30]:
merged_data_from_district = merged_data_from_district.sort_values('mean_discrepancy', ascending=False).head(10).reset_index(drop=True)

In [31]:
from math import pi, atan # returns radians
from folium.plugins import BeautifyIcon
def add_pairs_to_map_from_row(i, df_pairwise_with_geometry_filtered, m):
    lat1 = df_pairwise_with_geometry_filtered.loc[i]['from_geometry'].centroid.x
    lon1 = df_pairwise_with_geometry_filtered.loc[i]['from_geometry'].centroid.y
    lat2 = df_pairwise_with_geometry_filtered.loc[i]['to_geometry'].centroid.x
    lon2 = df_pairwise_with_geometry_filtered.loc[i]['to_geometry'].centroid.y
    from_district = df_pairwise_with_geometry_filtered.loc[i]['fromDistrict']
    to_district = df_pairwise_with_geometry_filtered.loc[i]['to_district']
    mean_discrepancy = df_pairwise_with_geometry_filtered.loc[i]['mean_discrepancy'].round(2)
    count_order = df_pairwise_with_geometry_filtered.loc[i]['count_order']
    lat_diff = lat2 - lat1
    lon_diff = lon2 - lon1
    rotation = pi*atan(lat_diff/lon_diff)
    folium.PolyLine(
        locations=[
            (lon1, lat1), (lon2, lat2)
        ],
        weight=5,
        color='blue',
        tooltip = f"""<b>Between:</b> {from_district}<br><br>
        <b>And:</b> {to_district}<br><br>
        """
    ).add_to(m)
    # folium.RegularPolygonMarker(location=(lat2, lon2), fill_color='yellow', number_of_sides=3, radius=10, rotation=rotation).add_to(m)
    icon_star = folium.Icon(
        prefix='fa', 
        icon='star', 
        icon_color='blue',
    )
    folium.Marker(
        location=(lon2-lon_diff/15, lat2-lat_diff/15),
        tooltip = f"""<b>From:</b> {from_district}<br><br>
        <b>To:</b> {to_district}<br><br>
        <b>Mean Discrepancy:</b> {mean_discrepancy}<br><br>
        <b>Count Order:</b> {count_order}
        """,
        icon=icon_star,
    ).add_to(m)

In [32]:
m = folium.Map(location=[10.7704143,106.6911201], tiles='cartodbpositron', zoom_start=12)
cp = folium.Choropleth(geo_data=merged_data.__geo_interface__, 
           data=merged_data['num_order'], 
#            columns=['city', 'cash_ratio'],
           key_on="feature.id", 
           fill_color='YlGnBu', 
           fill_opacity=0.3,
           legend_name='Number of Orders from this District'
          ).add_to(m)

for i in range(len(merged_data_from_district)):
    add_pairs_to_map_from_row(i, merged_data_from_district, m)

In [33]:
m